# task1b: EXTRA testing when applying directly the exact same OCR to different pdf conversion methods

We will apply pytesseract or layoutparser and then tesseract-ocr to see which one works better in a more real life scenario. This is much needed to see which one performs better in a more real life scenario.


## Index

1. [Introduction](#introduction)
2. [Setup](#setup)
    - [Install Dependencies](#install-dependencies)
    - [Environment Variables](#environment-variables)
3. [Methods](#methods)
    - [Pytesseract + pdf2image](#pytesseract--pdf2image)
    - [Pytesseract + pdfium](#pytesseract--pdfium)
    - [Pytesseract + PyMuPDF](#pytesseract--pymupdf)
    - [Pytesseract + Wand](#pytesseract--wand)
4. [Performance Check](#performance-check)
    - [Native Performance](#native-performance)
    - [Text Quality Check](#text-quality-check)
5. [Similarity Analysis](#similarity-analysis)

In [1]:
IRL_stats = []

In [2]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

## Pytesseract + pdf2image

In [3]:
%pip install pdf2image
%pip install pytesseract opencv-python pillow


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**RUN IN BASH TERMINAL AFTER PRESSING CTRL+SHIFT+P and writing terminal, press enter and click on bash**


__don't execute on python, open a bash terminal__
```bash
chmod +x ./install_tesseract.sh    #makes it executable
./install_tesseract.sh
```

In [4]:
import os
import time
import psutil
from pdf2image import convert_from_path
import pytesseract
###                                   TRACKING                                     ###
# Initialize variables
process = psutil.Process(os.getpid())
words = []

# Start measurements (proper CPU initialization)
_ = process.cpu_percent(interval=None)  # Dummy call to reset counter
mem_before = process.memory_info().rss
start_time = time.time()
###                                   TRACKING END                                 ###
# Process PDF
filePath = os.environ.get("filePath")
try:
    doc = convert_from_path(filePath,dpi=300)
    for page_number, page_data in enumerate(doc):
        try:
            txt = pytesseract.image_to_string(page_data)
            if txt:  # Only process if text was extracted
                words.extend(txt.split())
                print(f"Page #{page_number} - {txt}")
        except Exception as e:
            print(f"Error processing page {page_number}: {str(e)}")
except Exception as e:
    print(f"Error loading PDF: {str(e)}")

    
###                                   TRACKING                                     ###
# Calculate metrics
cpu_used = process.cpu_percent(interval=None)
memory_MB = process.memory_info().rss-mem_before
total_time = time.time() - start_time

# Store results
IRL_stats.append({
    "name": "tesseract+pdf2image",
    "time": total_time,
    "num_words": len(words),
    "memory_MB": memory_MB / (1024 * 1024),  # Delta memory
    "cpu_percent": cpu_used,
    "num_alphanum_chars": sum(1 for word in words for c in word if c.isalnum()),
})
###                                   TRACKING END                                 ###


Page #0 - Copie électronique

LABORATOIRE DE BIOLOGIE MEDICALE

 

  

N° FINESS : 

 -  -  @: 51  &: 
  - Biologiste(s) Médical(aux)

Docteur     Madame   
CABINET MEDICAL " " 
    (100)
Copie a : Docteur    , DR 
X Demande n° 01/02/ -LABO--TP Edité le, lundi 1 février 2021
Copie a : Docteur    , DR
Patient né(e)   le  

FSE Tiers payant  - 
Prélevements effectués par le laboratoire le 01/02/21 a 10H27

Vos résultats sur internet : Accés sécurisé, rapide, gratuit, pratique, @coresponsable
1) Communiquez votre mail au laboratoire 2) Recevez un email dés que vos résultats sont disponibles 3) Cliquez sur le lien

INFORMATION COVID-19
Rendez-vous sur notre site internet dédié pour connaitre notre organisation : https:// .fr/depistage-covid-19/

4 -
Hematologie
Valeurs de référence Antériorités
v Hémogramme
(Sang total - Variation d'impédance, photométrie, cytométrie en flux)  - 

HEMatieS ........cccecceceseeceeeeceeeeaeaeeaeees 4,94 Téra/L 3,80 a 5,90 4,97
HEMOGIODING ........ cece cece 

## Pytesseract + pdfium

In [5]:
%pip install pdfium2
%pip install pytesseract opencv-python pillow


ERROR: Could not find a version that satisfies the requirement pdfium2 (from versions: none)
ERROR: No matching distribution found for pdfium2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**RUN IN BASH TERMINAL AFTER PRESSING CTRL+SHIFT+P and writing terminal, press enter and click on bash**


__don't execute on python, open a bash terminal__
```bash
chmod +x ./install_tesseract.sh    #makes it executable
./install_tesseract.sh
```

In [6]:
import os
import time
import psutil
import pypdfium2 as pdfium
import pytesseract
###                                   TRACKING                                     ###
# Initialize variables
process = psutil.Process(os.getpid())
words2 = []

# Start measurements (proper CPU initialization)
_ = process.cpu_percent(interval=None)  # Dummy call to reset counter
mem_before = process.memory_info().rss
start_time = time.time()
###                                   TRACKING END                                 ###

# Process PDF
filePath = os.environ.get("filePath")
try:
    # Load a document
    pdf = pdfium.PdfDocument(filePath)

    # Pre-render all pages to avoid repeated calls
    rendered_pages = [page.render(scale=4).to_pil() for page in pdf]

    # Loop over pre-rendered pages and extract text
    for page_number, image in enumerate(rendered_pages):
        try:
            txt = pytesseract.image_to_string(image)
            if txt:  # Only process if text was extracted
                words2.extend(txt.split())
                print(f"Page #{page_number + 1} - {txt}")
        except Exception as e:
            print(f"Error processing page {page_number + 1}: {str(e)}")
except Exception as e:
    print(f"Error loading PDF: {str(e)}")
###                                   TRACKING                                     ###
# Calculate metrics
cpu_used = process.cpu_percent(interval=None)
memory_MB = process.memory_info().rss-mem_before
total_time = time.time() - start_time

# Store results
IRL_stats.append({
    "name": "tesseract+pdfium",
    "time": total_time,
    "num_words": len(words2),
    "memory_MB": memory_MB / (1024 * 1024),  # Delta memory
    "cpu_percent": cpu_used,
    "num_alphanum_chars": sum(1 for word in words2 for c in word if c.isalnum()),
})
###                                   TRACKING END                                 ###


Page #1 - Copie électronique

LABORATOIRE DE BIOLOGIE MEDICALE

 

  

N° FINESS : 

 -  -  @:  &:  44

  - Biologiste(s) Médical(aux)

Docteur    

Madame   

CABINET MEDICAL " " 
    (100)
Copie a : Docteur    , DR 
X Demande n° 01/02/ -LABO--TP Edité le, lundi 1 février 2021
Copie a : Docteur    , DR
Patient né(e)   le  

FSE Tiers payant  - 
Prélevements effectués par le laboratoire le 01/02/21 a 10H27

Vos résultats sur internet : Accés sécurisé, rapide, gratuit, pratique, @€coresponsable
1) Communiquez votre mail au laboratoire 2) Recevez un email dés que vos résultats sont disponibles 3) Cliquez sur le lien

INFORMATION COVID-19

Rendez-vous sur notre site internet dédié pour connaitre notre organisation : https:// .fr/depistage-covid-19/

Hématologie

v Hémogramme

(Sang total - Variation d'impédance, photométrie, cytométrie en flux)  - 

HE@MatieS ....ccccccccceceeeeeeeeeceevaueaneuaeeas 4,94 Téra/L
HEMOGIODING ........ccececececeaeeeeeeeeeeaeaees 13,6 g/dL
8,4 mmol/L
H@Matocr

## Pytesseract + PyMuPDF

In [7]:
%pip install PyMuPDF
%pip install pytesseract opencv-python pillow


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**RUN IN BASH TERMINAL AFTER PRESSING CTRL+SHIFT+P and writing terminal, press enter and click on bash**


__don't execute on python, open a bash terminal__
```bash
chmod +x ./install_tesseract.sh    #makes it executable
./install_tesseract.sh
```

In [8]:
import os
import time
import psutil
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
###                                   TRACKING                                     ###
# Initialize variables
process = psutil.Process(os.getpid())
words3 = []

# Start measurements (proper CPU initialization)
_ = process.cpu_percent(interval=None)  # Dummy call to reset counter
mem_before = process.memory_info().rss
start_time = time.time()
###                                   TRACKING END                                 ###

# Process PDF
filePath = os.environ.get("filePath")
try:
    doc = fitz.open(filePath)

    # Calculate scale matrix for 300 DPI (72 dpi is default)
    zoom = 300 / 72  # 4.1667 approx
    matrix = fitz.Matrix(zoom, zoom)

    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        pix = page.get_pixmap(matrix=matrix)
        try:
            # Convert pixmap to PIL image and extract text
            pil_image = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)
            txt = pytesseract.image_to_string(pil_image)
            if txt:  # Only process if text was extracted
                words3.extend(txt.split())
                print(f"Page #{page_number + 1} - {txt}")
        except Exception as e:
            print(f"Error processing page {page_number + 1}: {str(e)}")
except Exception as e:
    print(f"Error loading PDF: {str(e)}")
###                                   TRACKING                                     ###
# Calculate metrics
cpu_used = process.cpu_percent(interval=None)
memory_MB = process.memory_info().rss-mem_before
total_time = time.time() - start_time

# Store results
IRL_stats.append({
    "name": "tesseract+pymupdf",
    "time": total_time,
    "num_words": len(words3),
    "memory_MB": memory_MB / (1024 * 1024),  # Delta memory
    "cpu_percent": cpu_used,
    "num_alphanum_chars": sum(1 for word in words3 for c in word if c.isalnum()),
})
###                                   TRACKING END                                 ###


Page #1 - Copie électronique

LABORATOIRE DE BIOLOGIE MEDICALE

 

  

N° FINESS : 

 -  -  @: 51  &: 
  - Biologiste(s) Médical(aux)

Docteur     Madame   
CABINET MEDICAL " " 
    (100)
Copie a : Docteur    , DR 
X Demande n° 01/02/ -LABO--TP Edité le, lundi 1 février 2021
Copie a : Docteur    , DR
Patient né(e)   le  

FSE Tiers payant  - 
Prélevements effectués par le laboratoire le 01/02/21 a 10H27

Vos résultats sur internet : Accés sécurisé, rapide, gratuit, pratique, @€coresponsable
1) Communiquez votre mail au laboratoire 2) Recevez un email dés que vos résultats sont disponibles 3) Cliquez sur le lien

INFORMATION COVID-19
Rendez-vous sur notre site internet dédié pour connaitre notre organisation : https:// .fr/depistage-covid-19/

4 -
Hematologie
Valeurs de référence Antériorités
v Hémogramme
(Sang total - Variation d'impédance, photométrie, cytométrie en flux)  - 

HE@MatieS .....ccccccccccececeeeveeeeeaueeaueuuaes 4,94 Téra/L 3,80 a 5,90 4,97
HEMOGIODING ......e cece eee 

## Pytesseract + Wand

In [9]:
%pip install wand
%pip install pytesseract opencv-python pillow


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**RUN IN BASH TERMINAL AFTER PRESSING CTRL+SHIFT+P and writing terminal, press enter and click on bash**


__don't execute on python, open a bash terminal__
```bash
chmod +x ./install_tesseract.sh    #makes it executable
./install_tesseract.sh
```

In [10]:
import os
import time
import psutil
from wand.image import Image
from wand.color import Color
import pytesseract
import numpy as np
from PIL import Image as PILImage
###                                   TRACKING                                     ###
# Initialize variables
process = psutil.Process(os.getpid())
words4 = []

# Start measurements (proper CPU initialization)
_ = process.cpu_percent(interval=None)  # Dummy call to reset counter
mem_before = process.memory_info().rss
start_time = time.time()
###                                   TRACKING END                                 ###

# Process PDF
filePath = os.environ.get("filePath")
resolution = 300  # DPI for high quality
try:
    with Image(filename=filePath, resolution=resolution) as pdf:
        for page in pdf.sequence:
            with Image(page) as img:

                img.background_color = Color("white")  # Ensure a white background for transparency
                img.alpha_channel = 'remove'  # Remove alpha channel
                try:
                    img_pil=PILImage.fromarray(np.array(img))
                    # Extract text using pytesseract
                    txt = pytesseract.image_to_string(img_pil)
                    if txt:  # Only process if text was extracted
                        words4.extend(txt.split())
                        print(txt)
                except Exception as e:
                    print(f"Error processing page: {str(e)}")
except Exception as e:
    print(f"Error loading PDF: {str(e)}")
###                                   TRACKING                                     ###
# Calculate metrics
cpu_used = process.cpu_percent(interval=None)
memory_MB = process.memory_info().rss-mem_before
total_time = time.time() - start_time

# Store results
IRL_stats.append({
    "name": "tesseract+wand",
    "time": total_time,
    "num_words": len(words4),
    "memory_MB": memory_MB / (1024 * 1024),  # Delta memory
    "cpu_percent": cpu_used,
    "num_alphanum_chars": sum(1 for word in words4 for c in word if c.isalnum()),
})
###                                   TRACKING END                                 ###


Copie électronique

LABORATOIRE DE BIOLOGIE MEDICALE

 

  

N° FINESS : 

 -  -  @: 51  &:  53 15 44

  - Biologiste(s) Médical(aux)

Docteur    

Madame   

CABINET MEDICAL " " 
    (100)
Copie a : Docteur    , DR 
X Demande n° 01/02/ -LABO--TP Edité le, lundi 1 février 2021
Copie & : Docteur    , DR

Patient né(e)   le  

FSE Tiers payant  - 
Prélevements effectués par le laboratoire le 01/02/21 a 10H27

Vos résultats sur internet : Accés sécurisé, rapide, gratuit, pratique, e€coresponsable

1) Communiquez votre mail au laboratoire 2) Recevez un email dés que vos résultats sont disponibles

INFORMATION COVID-19

Rendez-vous sur notre site internet dédié pour connaitre notre organisation : https:// .fr/depistage-covid-19/

3) Cliquez sur le lien

Hématologie

v Hémogramme

(Sang total - Variation d'impédance, photométrie, cytométrie en flux)  - 

H@mMatieS .......ccccecseeseceeeeeeeeeeeueeuneeaes 4,94 Téra/L
H@MOGIODING ..........ccccceeeeeeeseeeeeeeeeeees 13,6 g/dL
8,4 mmol/L
H@Mato

### Native performance check:

Great way of measuring time and pictures taken but we shouldn't really trust memory and cpu usage in this method. Scalene is probably better.

In [11]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [12]:
from tabulate import tabulate

print(tabulate(IRL_stats, headers="keys"))


name                    time    num_words    memory_MB    cpu_percent    num_alphanum_chars
-------------------  -------  -----------  -----------  -------------  --------------------
tesseract+pdf2image  12.6698         2169     237.953             3.3                 11173
tesseract+pdfium     13.3272         2139     177.375            20.9                 11153
tesseract+pymupdf    15.045          2151      29.8281           11.6                 11117
tesseract+wand       19.4953         2128     341.492            30.1                 11219


### Text Quality Check:

[Methods](#methods)

- [Pytesseract + pdf2image](#pytesseract--pdf2image)
- [Pytesseract + pdfium](#pytesseract--pdfium)
- [Pytesseract + PyMuPDF](#pytesseract--pymupdf)
- [Pytesseract + Wand](#pytesseract--wand)

In [13]:
%pip install simphile

Note: you may need to restart the kernel to use updated packages.


In [14]:
from simphile import jaccard_similarity, compression_similarity, euclidian_similarity


pdf2image_text = ' '.join(words)
pdfium_text = ' '.join(words2)
pymupdf_text = ' '.join(words3)
wand_text = ' '.join(words4)
# Load the text from the highlighted_output.txt file

with open("highlighted_output.txt", "r") as file:
    txt_doc_text = file.read()
# Calculate similarities
similarities = {
    "pdf2image": {
        "jaccard": jaccard_similarity(txt_doc_text, pdf2image_text),
        "compression": compression_similarity(txt_doc_text, pdf2image_text),
        "euclidean": euclidian_similarity(txt_doc_text, pdf2image_text),
    },
    "pdfium": {
        "jaccard": jaccard_similarity(txt_doc_text, pdfium_text),
        "compression": compression_similarity(txt_doc_text, pdfium_text),
        "euclidean": euclidian_similarity(txt_doc_text, pdfium_text),
    },
    "pymupdf": {
        "jaccard": jaccard_similarity(txt_doc_text, pymupdf_text),
        "compression": compression_similarity(txt_doc_text, pymupdf_text),
        "euclidean": euclidian_similarity(txt_doc_text, pymupdf_text),
    },
    "wand": {
        "jaccard": jaccard_similarity(txt_doc_text, wand_text),
        "compression": compression_similarity(txt_doc_text, wand_text),
        "euclidean": euclidian_similarity(txt_doc_text, wand_text),
    },
}
# Print similarities
for retriever, metrics in similarities.items():
    print(f"Retriever: {retriever}")
    for metric, value in metrics.items():
        print(f"  {metric.capitalize()} Similarity: {value:.4f}")

# Calculate the average similarity score for each method
average_similarity_scores = {
    retriever: sum(metrics.values()) / len(metrics)
    for retriever, metrics in similarities.items()
}

# Find the method with the highest average similarity score
most_similar_method = max(average_similarity_scores, key=average_similarity_scores.get)

# Print the results
print("\nAverage Similarity Scores:")
for retriever, score in average_similarity_scores.items():
    print(f"{retriever}: {score:.4f}")

print(f"\nThe method that returns the most similar text is: {most_similar_method}")


Retriever: pdf2image
  Jaccard Similarity: 0.6643
  Compression Similarity: 0.6334
  Euclidean Similarity: 0.9827
Retriever: pdfium
  Jaccard Similarity: 0.6622
  Compression Similarity: 0.6361
  Euclidean Similarity: 0.9824
Retriever: pymupdf
  Jaccard Similarity: 0.6745
  Compression Similarity: 0.6358
  Euclidean Similarity: 0.9826
Retriever: wand
  Jaccard Similarity: 0.6631
  Compression Similarity: 0.6289
  Euclidean Similarity: 0.9834

Average Similarity Scores:
pdf2image: 0.7601
pdfium: 0.7602
pymupdf: 0.7643
wand: 0.7585

The method that returns the most similar text is: pymupdf
